In [69]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/sandeep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sandeep/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Query

In [71]:
class Response():
    def __init__(self,data):
        self.data=set(data)

    def getMapping(self,file):
        self.mapping=json.load(open(file))
        for i in self.data:
            print(self.mapping[i])
    
    def __str__(self):
        return str(len(self.data))

    def add(self,response):
        return Response(set.union(self.data,response.data))
        
    def intersect(self,response):
        return Response(set.intersection(self.data,response.data))

    def diff(self,response):
        return Response(set.difference(self.data,response.data))
        

In [72]:
class Query():
    def __init__(self,file):
        self.db=json.load(open(file))
        self.db=defaultdict(lambda:[],self.db)        

    def OR(self,term1,term2):
        return term1.add(term2)
    
    def AND(self,term1,term2):
        return term1.intersect(term2)
    
    def ANDNOT(self,term1,term2):
        # temp=Response(set(sum(list(self.db.copy().values()),[])))
        temp=Response(np.arange(468))
        temp1=temp.diff(term2)
        return term1.intersect(temp1)

    def ORNOT(self,term1,term2):
        # temp=Response(set(sum(list(self.db.copy().values()),[])))
        temp=Response(np.arange(468))
        temp1=temp.diff(term2)
        temp2 = term1.intersect(term2)
        return temp1.add(temp2)

    def no_comparisonsOR(self,term1, term2):
        first = self.db[term1]
        second =self.db[term2]
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                ans.append(first[i])
                i+=1          
            elif(first[i]>second[j]):
                ans.append(second[j])
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        
        ans.extend(first[i:])
        ans.extend(second[j:])

        return count    

    def no_comparisonsAND(self,term1, term2):
        first = self.db[term1]
        second =self.db[term2]
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                i+=1          
            elif(first[i]>second[j]):
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        return count
    
    def no_comparisonsANDNOT(self,term1, term2):
        temp=Response(np.arange(468)) 
        second = Response(self.db[term2]) #list of term2
        second=temp.diff(second) #generating Universal - (Not terms2)
        second=list(second.data)
        second.sort() #sorting, if needed
        first = list(set(self.db[term1])) #list of term1
        first.sort()
        #initiating general AND operator
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                i+=1          
            elif(first[i]>second[j]):
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        return count
    
    def no_comparisonsORNOT(self,term1, term2):
        temp=Response(np.arange(468)) 
        second = Response(self.db[term2]) #list of term2
        second=temp.diff(second) #generating Universal - (Not terms2)
        second=list(second.data)
        second.sort() #sorting, if needed
        first = self.db[term1] #list of term1
    
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                ans.append(first[i])
                i+=1          
            elif(first[i]>second[j]):
                ans.append(second[j])
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        while(i<len(first)):
            ans.append(first[i])
            i+=1
        while(j<len(second)):
            ans.append(second[j])
            j+=1
        return count  


In [73]:
query=Query("output.json")

In [76]:
queries=["stink","middle"]

output=Response(query.db[queries[0]])

for i in tnrange(1,len(queries)):
    curr=Response(query.db[queries[i]])
    output=query.OR(curr,output)
    print(query.no_comparisonsAND(queries[i],queries[i-1]))

print(output)

  0%|          | 0/1 [00:00<?, ?it/s]

19
25


In [64]:
output.getMapping("mapping.json")
output.data

stories/grav
stories/quest
stories/bumm
stories/rocket.sf
stories/deathmrs.d
stories/zombies.txt
stories/gulliver.txt
stories/darkness.txt
stories/crazy.hum
stories/hellmach.txt
stories/fic1
stories/long1-3.txt
stories/elite.app
stories/nigel.5
stories/forgotte
stories/nihgel_8.9
stories/aesop11.txt
stories/sick-kid.txt
stories/retrib.txt
stories/keepmodu.txt
stories/beggars.txt
stories/girlclub.txt
stories/friends.txt
stories/hitch2.txt
stories/valen


{0,
 24,
 30,
 64,
 112,
 132,
 143,
 145,
 155,
 161,
 164,
 179,
 202,
 228,
 232,
 251,
 266,
 268,
 287,
 298,
 300,
 323,
 337,
 371,
 439}